In [ ]:
import os
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from joblib import load

# 결과 저장 디렉토리
output_dir = "../results/reports"
os.makedirs(output_dir, exist_ok=True)

# 모델 저장된 디렉토리
model_dir = "../results/models"

# 데이터 로드
df = pd.read_csv("../data/processed/data_cleaned.csv")

# X와 y로 분리 (y는 'Label' 컬럼)
X = df.drop(columns=['Label'])
y = df['Label']

# 훈련/테스트 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델, 스케일러, PCA 종류
scalers = [None, MinMaxScaler(), StandardScaler()]
pca_components = [None, 0.99, 10]
models = ['rf', 'xgb', 'catboost', 'lightgbm']

# 성능 결과 저장
results = []

# 저장된 모델을 불러와 성능 평가
for scaler in scalers:
    for pca in pca_components:
        for model_name in models:
            # 모델 파일 이름
            scaler_name = str(scaler).split()[0] if scaler else "None"
            pca_name = str(pca)
            model_filename = f"{model_name}_scaler_{scaler_name}_pca_{pca_name}.joblib"
            model_path = os.path.join(model_dir, model_filename)
            
            # 모델 파일 존재 여부 확인
            if not os.path.exists(model_path):
                print(f"모델 파일을 찾을 수 없습니다: {model_path}")
                continue
            
            # 스케일러 적용 (스케일러가 None이 아닌 경우에만)
            if scaler:
                # 훈련 데이터에 fit 후 변환, 테스트 데이터는 transform만 적용
                X_train_scaled = scaler.fit_transform(X_train)
                X_test_scaled = scaler.transform(X_test)
            else:
                X_train_scaled = X_train
                X_test_scaled = X_test
            
            # PCA 적용 (PCA가 None이 아닌 경우에만)
            if pca:
                pca_model = PCA(n_components=pca)
                X_train_scaled = pca_model.fit_transform(X_train_scaled)
                X_test_scaled = pca_model.transform(X_test_scaled)
            
            # 모델 로드
            print(f"모델 로드 중: {model_path}")
            pipeline = load(model_path)
            
            # 테스트 데이터 예측
            y_pred = pipeline.predict(X_test_scaled)
            y_pred_proba = pipeline.predict_proba(X_test_scaled)[:, 1] if hasattr(pipeline, 'predict_proba') else None
            
            # 성능 평가
            acc = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='weighted')
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            
            # classification_report 생성 (텍스트 형식)
            class_report = classification_report(y_test, y_pred, output_dict=True)
            
             # classification_report를 텍스트 파일로 저장
            report_filename = f"{model_name}_scaler_{scaler_name}_pca_{pca_name}_classification_report.txt"
            report_path = os.path.join(output_dir, report_filename)
            
            with open(report_path, "w") as f:
                f.write(f"Classification Report for {model_name} with Scaler: {scaler_name} and PCA: {pca_name}\n")
                f.write(classification_report(y_test, y_pred))

            # 결과 저장
            results.append({
                'Model': model_name,
                'Scaler': scaler_name,
                'PCA': pca_name,
                'Accuracy': acc,
                'F1 Score': f1,
                'MSE': mse,
                'R2': r2,
                'Classification Report': class_report  # 세부 보고서
            })

# 결과 출력
results_df = pd.DataFrame([{k: v for k, v in res.items() if k != 'Classification Report'} for res in results])
print(results_df)

# 결과 CSV로 저장
results_df.to_csv(os.path.join(output_dir, "classification_scores.csv"), index=False)

# 5m36.3s


모델 로드 중: ../results/models\rf_scaler_None_pca_None.joblib
모델 로드 중: ../results/models\xgb_scaler_None_pca_None.joblib


c:\Users\dev\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\dev\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\dev\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\dev\anaconda3\Lib\site-packages\sklea

모델 로드 중: ../results/models\catboost_scaler_None_pca_None.joblib
모델 로드 중: ../results/models\lightgbm_scaler_None_pca_None.joblib
모델 로드 중: ../results/models\rf_scaler_None_pca_0.99.joblib


c:\Users\dev\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\dev\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\dev\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\dev\anaconda3\Lib\site-packages\sklea

모델 로드 중: ../results/models\xgb_scaler_None_pca_0.99.joblib


...

모델 로드 중: ../results/models\lightgbm_scaler_StandardScaler()_pca_None.joblib
모델 로드 중: ../results/models\rf_scaler_StandardScaler()_pca_0.99.joblib
모델 로드 중: ../results/models\xgb_scaler_StandardScaler()_pca_0.99.joblib
모델 로드 중: ../results/models\catboost_scaler_StandardScaler()_pca_0.99.joblib


c:\Users\dev\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\dev\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\dev\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\dev\anaconda3\Lib\site-packages\sklea

모델 로드 중: ../results/models\lightgbm_scaler_StandardScaler()_pca_0.99.joblib
모델 로드 중: ../results/models\rf_scaler_StandardScaler()_pca_10.joblib
모델 로드 중: ../results/models\xgb_scaler_StandardScaler()_pca_10.joblib
모델 로드 중: ../results/models\catboost_scaler_StandardScaler()_pca_10.joblib


c:\Users\dev\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\dev\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\dev\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\dev\anaconda3\Lib\site-packages\sklea

모델 로드 중: ../results/models\lightgbm_scaler_StandardScaler()_pca_10.joblib
       Model            Scaler   PCA  Accuracy  F1 Score        MSE        R2
0         rf              None  None  0.998514  0.998479   0.157001  0.990973
1        xgb              None  None  0.998903  0.998864   0.109673  0.993694
2   catboost              None  None  0.998076  0.998025   0.154833  0.991097
3   lightgbm              None  None  0.975450  0.974772   1.551995  0.910763
4         rf              None  0.99  0.994485  0.994414   0.414072  0.976191
5        xgb              None  0.99  0.985250  0.984942   1.180289  0.932135
6   catboost              None  0.99  0.926235  0.926591   6.954089  0.600150
7   lightgbm              None  0.99  0.703762  0.743410  34.617436 -0.990451
8         rf              None    10  0.998467  0.998419   0.162169  0.990676
9        xgb              None    10  0.998732  0.998666   0.127860  0.992648
10  catboost              None    10  0.995406  0.995180   0.366465 

In [ ]:
import os
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, f1_score, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from joblib import load
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler

# 결과 저장 디렉토리
output_dir = "../results/reports"
os.makedirs(output_dir, exist_ok=True)

# 모델 저장된 디렉토리
model_dir = "../results/models"

# 데이터 로드
df = pd.read_csv("../data/processed/data_cleaned.csv")

# X와 y로 분리 (y는 'Label' 컬럼)
X = df.drop(columns=['Label'])
y = df['Label']

# 훈련/테스트 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 이름 설정
models = ['autoencoder_classifier', 'random_forest']

# 성능 결과 저장
results = []

# 저장된 모델을 불러와 성능 평가
for model_name in models:
    # 모델 파일 이름
    if model_name == 'autoencoder_classifier':
        model_filename = "autoencoder_classifier.keras"
        model_path = os.path.join(model_dir, model_filename)
        model = load_model(model_path)
        
        # Autoencoder 모델에서 분류기 부분만 사용
        encoder = Model(inputs=model.input, outputs=model.get_layer('dense_4').output)
        X_train_encoded = encoder.predict(X_train_scaled)  # 스케일링된 데이터로 예측
        X_test_encoded = encoder.predict(X_test_scaled)    # 스케일링된 데이터로 예측

        # 분류기 평가
        y_pred_binary = (model.predict(X_test_scaled)[1] > 0.5).astype(int)  # 스케일링된 데이터로 예측

    elif model_name == 'random_forest':
        model_filename = "autoencoder_random_forest.pkl"
        model_path = os.path.join(model_dir, model_filename)
        model = load(model_path)

        # Random Forest 모델 평가
        y_pred_binary = model.predict(X_test_encoded)  # 인코딩된 데이터를 사용하여 예측
    
    # 성능 평가
    acc = accuracy_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary, average='weighted')
    mse = mean_squared_error(y_test, y_pred_binary)
    r2 = r2_score(y_test, y_pred_binary)
    
    # classification_report 생성 (텍스트 형식)
    class_report = classification_report(y_test, y_pred_binary, output_dict=True)
    
    # classification_report를 텍스트 파일로 저장
    report_filename = f"{model_name}_classification_report.txt"
    report_path = os.path.join(output_dir, report_filename)
    with open(report_path, "w") as f:
        f.write(f"Classification Report for {model_name}\n")
        f.write(classification_report(y_test, y_pred_binary))

    # 결과 저장
    results.append({
        'Model': model_name,
        'Accuracy': acc,
        'F1 Score': f1,
        'MSE': mse,
        'R2': r2,
        'Classification Report': class_report  # 세부 보고서
    })

# 결과 출력
results_df = pd.DataFrame([{k: v for k, v in res.items() if k != 'Classification Report'} for res in results])
print(results_df)

# 결과 CSV로 저장
results_df.to_csv(os.path.join(output_dir, "classification_scores.csv"), index=False)
